# Proscenium Legal Demo

This notebook demonstrates

1. The use of an LLM to enrich of publically available legal opinions
2. Entity resolution
3. Query parsing
4. Context formation for question answering

## Prerequisites

1. Obtain a `TOGETHER_API_KEY` from https://together.ai/ and store it as an environment variable or Colab secret.
2. Obtain a Neo4j Sandbox https://neo4j.com/sandbox/



## Setup

In [1]:
!git clone https://github.com/The-AI-Alliance/proscenium.git

Cloning into 'proscenium'...
remote: Enumerating objects: 1081, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 1081 (delta 234), reused 198 (delta 171), pack-reused 745 (from 2)
Receiving objects: 100% (1081/1081), 2.29 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (618/618), done.


In [2]:
%cd proscenium

/content/proscenium


In [3]:
!python -m pip install .

Processing /content/proscenium
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.4/437.4 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
from rich import print
from rich.panel import Panel
from rich.prompt import Prompt
import asyncio

import nest_asyncio
nest_asyncio.apply()

## Configure

The folowing cell defines

1. Get secrets for accessing Together and Neo4j
2. Hugging Face dataset id (NH caselaw)
3. Filenames (entity csv)
4. Model ids (for extraction, embedding, and generation)
5. Domain-specific logic for processing caselaw and user queries



In [6]:
default_neo4j_uri = "bolt://localhost:7687"
default_neo4j_username = "neo4j"
default_neo4j_password = "password"

default_milvus_uri = "file:/grag-milvus.db"

try:
    from google.colab import userdata
    import os
    api_key = userdata.get('TOGETHER_API_KEY')
    os.environ['TOGETHER_API_KEY'] = api_key
    print("Pulled secrets from colab userdata")

    milvus_uri = default_milvus_uri
    milvus_uri = userdata.get("MILVUS_URI")

    neo4j_uri = default_neo4j_uri
    neo4j_uri = userdata.get('NEO4J_URI')

    neo4j_username = default_neo4j_username
    neo4j_username = userdata.get('NEO4J_USERNAME')

    neo4j_password = default_neo4j_password
    neo4j_password = userdata.get('NEO4J_PASSWORD')

except ImportError:
    print("Not in colab.  Relying on environment variables")
    milvus_uri = os.environ.get("MILVUS_URI", default_milvus_uri)
    neo4j_uri = os.environ.get("NEO4J_URI", default_neo4j_uri)
    neo4j_username = os.environ.get("NEO4J_USERNAME", default_neo4j_username)
    neo4j_password = os.environ.get("NEO4J_PASSWORD", default_neo4j_password)


from pathlib import Path
enrichment_jsonl_file = Path("enrichments.jsonl")

milvus_uri = "file:/grag-milvus.db"

import demo.domains.legal as domain

Pulled secrets from colab userdata

TypeError: get() takes 1 positional argument but 2 were given

## Extract Knowledge Graph

In [ ]:
from proscenium.scripts.document_enricher import enrich_documents

extract_from_opinion_chunks = domain.extract_from_opinion_chunks_function(
    domain.doc_as_rich,
    domain.default_chunk_extraction_model_id,
    domain.chunk_extraction_template,
    domain.LegalOpinionChunkExtractions,
    delay=0.1,
)

docs_per_dataset = 4

enrich_documents(
    domain.retriever(docs_per_dataset),
    extract_from_opinion_chunks,
    domain.doc_enrichments,
    enrichment_jsonl_file,
    verbose=True,
)

## Load Knowledge Graph

In [ ]:
from proscenium.verbs.know import knowledge_graph_client
from proscenium.scripts.knowledge_graph import load_knowledge_graph

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

load_knowledge_graph(
    driver,
    enrichment_jsonl_file,
    domain.LegalOpinionEnrichments,
    domain.doc_enrichments_to_graph,
)

driver.close()


## Show Knowledge Graph

In [ ]:
driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

domain.show_knowledge_graph(driver)

driver.close()

## Load Entity Resolver

In [ ]:
from proscenium.scripts.entity_resolver import load_entity_resolver

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

load_entity_resolver(
    driver,
    domain.resolvers,
    milvus_uri,
)

driver.close()

## Answer Question

In [ ]:
question = "How has 291 A.2d 605 been used in NH caselaw?"

In [ ]:
from proscenium.scripts.graph_rag import query_to_prompts
from rich.panel import Panel

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

In [ ]:
prompts = query_to_prompts(
    question,
    domain.default_query_extraction_model_id,
    milvus_uri,
    driver,
    domain.query_extract,
    domain.query_extract_to_graph,
    domain.query_extract_to_context,
    domain.context_to_prompts,
    verbose=True,
)


In [ ]:
from proscenium.verbs.complete import complete_simple

if prompts is None:

    print("Unable to form prompts")

else:

    system_prompt, user_prompt = prompts

    response = complete_simple(
        domain.default_generation_model_id,
        system_prompt,
        user_prompt,
        rich_output=True,
    )

    if response:
        print(Panel(response, title="Answer"))
    else:
        print("No answer")

driver.close()